In [ ]:
import pickle, os

import matplotlib.pyplot as plt
import numpy as np 
import random

from sklearn.linear_model import LogisticRegression

In [ ]:
def flatten(t):
    return np.array([item for sublist in t for item in sublist])

In [ ]:
def sigmoid(w,b,x): 
    return 1 / (1 + np.exp(-w*x+b) ) 

def update_rule(variable, result, rule):
    

    reward = -1
    failure = 1
    
    if rule == 'stimulus':
        
        if result == True:
            result_value = reward 
        
        elif result == False:
            result_value = failure
            
        variable += result_value 
    
    elif rule == 'inference':
        
        if result == True:
            result_value = reward 
            variable = 0
            
        elif result == False:
            result_value = failure
            variable += result_value
            
    return variable, result_value

In [ ]:
weight = #specify vluae
bias = # specify value


dataset_number = #specify value

nBouts = 250

bout = []

noise_var = 0.0



while len(bout) < nBouts:

    pReward = 0.9
    pSwitch = 0.3

    switch = 0#False
    move = False

    reward = -1
    failure = 1
    variable = 0

    myinputs = []
    myvariable = [variable]

    switch_result_check = [switch]
    moving_probability = [0]

    while not move:
        result = np.random.binomial(1, pReward)
        
        variable, result_value = update_rule(variable, result, rule = 'stimulus') # 'stimulus' 'inference'

        myinputs.append(result_value)
        myvariable.append(variable)
        
        switch_result_check.append(switch)

        if switch == False:
            switch = np.random.binomial(1,pSwitch)
        elif switch == True:
            pReward = 0

        pMove = sigmoid(weight,bias, variable)
        moving_probability.append(pMove)

        move = np.random.binomial(1, pMove)
        
        if move == True:
            break

    if len(myinputs) >= 1:
        bout.append([myvariable, myinputs ,switch_result_check, moving_probability])

In [ ]:
"""
Here we will attempt to divide the test/train data per bout vs
randomly sampling from the entire dataset
"""

percentTestBouts = 0.1

numBouts = int(nBouts*percentTestBouts)

testBoutsIdx = [random.randint(0, nBouts-1) for i in range(numBouts)]

#variable = bout[boutNumber][0]
mouse_variable = []
mouse_state = []

for boutNumber in range(nBouts):
    state = np.zeros(len(bout[boutNumber][0]))
    state[-1] = 1
    
    mouse_state.append(state)
    mouse_variable.append(bout[boutNumber][0])
    
flatten_variable = flatten(mouse_variable)
flatten_state = flatten(mouse_state)

mouse_variable_np = np.array(mouse_variable)
mouse_state_np = np.array(mouse_state)

x_test = mouse_variable_np[testBoutsIdx]
y_test = mouse_state_np[testBoutsIdx]

x_test,y_test = flatten(x_test) , flatten(y_test)

x_train = np.delete(mouse_variable_np,testBoutsIdx)
y_train = np.delete(mouse_state_np,testBoutsIdx)

x_train,y_train = flatten(x_train),flatten(y_train)

x_train = x_train.reshape((-1,1))
x_test = x_test.reshape((-1,1))

In [ ]:
model = LogisticRegression()
# model.fit(x_train.reshape((-1,1)), y_train)
model.fit(x_train, y_train)


In [ ]:
#model.predict(x_test)
test_accuracy = model.score(x_test, y_test)

weight_predict = (model.coef_).flatten()
bias_predict = -(model.intercept_).flatten()
probs = model.predict_proba(x_test)

In [ ]:
plt.figure( figsize = (15,8))

x_min = -50
x_max = -x_min
x_axis = np.arange(x_min,x_max,0.1)

plt.plot(x_test,y_test, '.',alpha = 0.4,markersize=12, color = 'blue', label = 'Binary Decisions')
plt.plot(x_test,probs.T[1,:], 'x',alpha = 0.4,markersize=12, color = 'red', label = 'Predicted Probabilities')

plt.plot(x_axis, sigmoid(weight,bias,x_axis), linewidth = 5, color = 'green', label = 'Original Sigmoid Function')
plt.plot(x_axis, sigmoid(weight_predict,bias_predict,x_axis), '-.' , linewidth = 2,color = 'orange', label = 'Predicted Sigmoid Function')

plt.text(-53, 0.6, f'$\eta = \omega x + b +\epsilon$\n$\omega_p = ${np.round(weight_predict[0],2)}\n$b_p = ${np.round(bias_predict[0],2)}', fontsize = 17)

plt.text(-53, 0.5, f'Model Accuracy: {np.round(test_accuracy*100,2)} %', fontsize = 17)


plt.legend( fontsize = 15)

plt.title(f'Stimulus Strategy\n$\omega = ${np.round(weight,2)} $b = ${np.round(bias,2)}', fontsize = 25)

plt.xlabel('Value Parameter',  fontsize = 20)
plt.ylabel('Move Probability',  fontsize = 20)

plt.show()

In [ ]:



export_data = [[x_train,y_train],[x_test,y_test]]

export_path = "./data/"+ str(dataset_number) +"/"
if not os.path.exists(export_path):
    os.makedirs(export_path)

export_filename = "data"
outfile = open(export_path+export_filename,'wb')
pickle.dump(export_data,outfile)
outfile.close()